In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename
sys.path.append(str(Path(getcwd()).parent.parent))
from epiweeks import Week
from glob import glob

In [ ]:
from bulletin import default_input, default_output, tables_path, hoje
from bulletin.systems import Notifica, CasosConfirmados, eSUS
from bulletin.utils.normalize import normalize_cpf, normalize_cns, normalize_hash, date_hash
from bulletin import default_input, default_output

In [ ]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from datetime import date, timedelta, datetime

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:
from bulletin import default_input, default_output
default_input = join(default_input,'omicron')
default_output = join(default_output,'omicron')

In [ ]:
notifica = Notifica('notifica')
notifica.load(compress=False)

In [ ]:
gal_path = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')

gal_2020 = pd.read_pickle(join(gal_path, 'banco_gal_2020-bz2.pkl'), compression='bz2')[['requisicao','cod_exame_requisicao','data_requisicao', 'paciente', 'cns', 'cpf', 'nome_mae','data_nascimento', 'sexo', 'idade', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]
gal_2021 = pd.read_pickle(join(gal_path, 'banco_gal_2021-bz2.pkl'), compression='bz2')[['requisicao','cod_exame_requisicao','data_requisicao', 'paciente', 'cns', 'cpf', 'nome_mae','data_nascimento', 'sexo', 'idade', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]
gal_2021_complemento = pd.read_csv(join(gal_path, 'consulta-gal-complemento-2021.csv'), sep = ';', low_memory=False, usecols=['requisicao','cod_exame_requisicao','data_requisicao', 'paciente', 'cns', 'cpf', 'nome_mae', 'data_nascimento','sexo', 'idade', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao'])
gal = pd.concat([gal_2020, gal_2021,gal_2021_complemento])
# gal.set_index('requisicao', inplace=True)

In [ ]:
gal = gal.loc[gal['virus']=='Coronavírus SARS-CoV2']
gal = gal.loc[gal['classificacao']=='Confirmado']

In [ ]:
gal['cpf'] = gal['cpf'].apply(normalize_cpf)
gal['cns'] = gal['cns'].apply(normalize_cns)
gal['hash_mae'] = gal['paciente'].apply(normalize_hash) + gal['nome_mae'].apply(normalize_hash)

In [ ]:
nomes = pd.read_excel(join(default_input, 'nomes_omicron.xlsx'))
nomes['nascimento'] = pd.to_datetime(nomes['nascimento'], errors='coerce')
nomes['hash_nasc'] = nomes.loc[nomes['nascimento'].notna(),'paciente'].apply(normalize_hash) + nomes.loc[nomes['nascimento'].notna(),'nascimento'].apply(date_hash)
nomes

In [ ]:
achados_gal = pd.merge(nomes, gal, left_on='requisicao_gal', right_on='requisicao', how='inner')
achados_gal = achados_gal.drop(columns={'paciente_y'})
achados_gal = achados_gal.rename(columns={'paciente_x':'paciente'})
achados_gal

In [ ]:
achados_hash_mae = pd.merge(achados_gal, notifica.df[['id','cep_residencia','logradouro_residencia','numero_residencia','bairro_residencia','telefone_paciente','data_1o_sintomas','hospitalizado','tipo_internacao','historico_viagem','local_viagem','data_ida_local','data_retorno_local','classificacao_final','criterio_classificacao','evolucao','data_cura_obito','hash_mae']], left_on='hash_mae', right_on='hash_mae', how='inner')
achados_hash_mae

In [ ]:
temporario1 = achados_hash_mae.drop_duplicates(subset='id', keep='last')
temporario1 = temporario1.drop_duplicates(subset='requisicao', keep='last')
temporario1

In [ ]:
achados_hash_nasc = pd.merge(achados_gal, notifica.df[['id','cep_residencia','logradouro_residencia','numero_residencia','bairro_residencia','telefone_paciente','data_1o_sintomas','hospitalizado','tipo_internacao','historico_viagem','local_viagem','data_ida_local','data_retorno_local','classificacao_final','criterio_classificacao','evolucao','data_cura_obito','hash_nasc']], left_on='hash_nasc', right_on='hash_nasc', how='inner')
achados_hash_nasc

In [ ]:
temporario2 = achados_hash_nasc.drop_duplicates(subset='id', keep='last')
temporario2 = temporario2.drop_duplicates(subset='requisicao', keep='last')
temporario2

In [ ]:
temporario = pd.concat([temporario1, temporario2], ignore_index=True)
temporario = temporario.drop_duplicates(subset='id', keep='last')
temporario = temporario.drop_duplicates(subset='requisicao', keep='last')
temporario

In [ ]:
temporario.loc[temporario['classificacao_final'] == 1, 'classificacao_final'] = "Caso suspeito"
temporario.loc[temporario['classificacao_final'] == 2, 'classificacao_final'] = "Caso confirmado"
temporario.loc[temporario['classificacao_final'] == 3, 'classificacao_final'] = "Caso descartado"

temporario.loc[temporario['criterio_classificacao'] == 1, 'criterio_classificacao'] = "Laboratorial"
temporario.loc[temporario['criterio_classificacao'] == 2, 'criterio_classificacao'] = "Clínico epidemiologico"
temporario.loc[temporario['criterio_classificacao'] == 3, 'criterio_classificacao'] = "Em investigacao"
temporario.loc[temporario['criterio_classificacao'] == 4, 'criterio_classificacao'] = "Não se aplica"
temporario.loc[temporario['criterio_classificacao'] == 5, 'criterio_classificacao'] = "Clínico imagem"
temporario.loc[temporario['criterio_classificacao'] == 6, 'criterio_classificacao'] = "Clínico"



temporario.loc[temporario['evolucao'] == 1, 'evolucao'] = "Cura"
temporario.loc[temporario['evolucao'] == 2, 'evolucao'] = "Obito por COVID-19"
temporario.loc[temporario['evolucao'] == 3, 'evolucao'] = "Não se aplica"
temporario.loc[temporario['evolucao'] == 4, 'evolucao'] = "Obito por outras causas"
temporario.loc[temporario['evolucao'] == 5, 'evolucao'] = "Obito por COVID-19"


In [ ]:
notifica_temp = Notifica()
notifica_temp.df = temporario.copy()
notifica_temp.df

In [ ]:
# notifica_temp.toggle_merge_table('hospitalizado')
# notifica_temp.toggle_merge_table('tipo_internacao')
# notifica_temp.toggle_merge_table('historico_viagem')
notifica_temp.toggle_merge_table('classificacao_final')
notifica_temp.toggle_merge_table('criterio_classificacao')
notifica_temp.toggle_merge_table('evolucao')

In [ ]:
notifica_temp.df

In [ ]:
temporario.to_excel(join(default_output, 'resultado_rastreio_omicron.xlsx'))